In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LogAnalysis") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/13 12:23:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/13 12:23:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Generar logs sintéticos

In [2]:
from src.log_generator import generate_logs

# Número de filas a generar (lo podés cambiar y volver a correr)
n_rows = 500000  
output_file = "../data/synthetic_logs.log"

generate_logs(n_rows, output_file)


ModuleNotFoundError: No module named 'src'

## Leer y parsear los logs

In [ ]:
raw_logs = spark.read.text("../data/synthetic_logs.log")
raw_logs.show(5, truncate=False)


In [ ]:
from pyspark.sql.functions import regexp_extract

log_df = raw_logs.select(
    regexp_extract("value", r'^(\S+)', 1).alias("ip"),
    regexp_extract("value", r'\[(.*?)\]', 1).alias("timestamp"),
    regexp_extract("value", r'\"(GET|POST) (.*?) HTTP.*\"', 2).alias("url"),
    regexp_extract("value", r'\" (GET|POST) (.*?) HTTP.*\"', 1).alias("method"),
    regexp_extract("value", r'\".*\" (\d{3})', 1).cast("int").alias("status"),
    regexp_extract("value", r'\".*\" \d{3} (\d+)', 1).cast("int").alias("bytes")
)

log_df.show(5, truncate=False)


### Procesamiento de timestamp, fecha y hora

In [ ]:
from pyspark.sql.functions import to_timestamp

log_df = log_df.withColumn(
    "datetime",
    to_timestamp("timestamp", "dd/MMM/yyyy:HH:mm:ss")
)


In [ ]:
from pyspark.sql.functions import hour, date_format

log_df = log_df.withColumn("hour", hour("datetime"))
log_df = log_df.withColumn("date", date_format("datetime", "yyyy-MM-dd"))


## Análisis exploratorio básico